In [1]:
import sys
sys.path.append('../')
from modules.biorreactor import BioReactor
import ipywidgets as wg
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# definition of controls
controls = dbc.Card(
    [
        dbc.FormGroup([
            dbc.Label('Produção Anual de Etanol [m³]:'),
            dbc.Input(id='prod_value', type='number', value=1000)
        ]),
        dbc.FormGroup([
            dbc.Label('Constante de Monod - Ks [g/dm³]:'),
            dcc.Slider(id='ks_value', min = 0.85, max = 6, step=0.05, value=1.7,
                      marks={
                          0.85: '0.85',
                          1.7: '1.70',
                          3.4: '3.40',
                          5.1: '5.10'
                      }),
            html.Div(id='K_s_slider_output')
        ]),
        dbc.FormGroup([
            dbc.Label(r'Velocidade Máxima - u (max) [1/h]:'),
            dcc.Slider(id='mu_value', min = 0.01, max = 1.33, step=0.01, value=0.33,
                      marks={
                          0.11: '0.11',
                          0.33: '0.33',
                          0.66: '0.66',
                          0.99: '0.99'
                      }),
            html.Div(id='mu_slider_output')
        ]),
        dbc.FormGroup([
            dbc.Label(r'Concentração Inicial Células - Cc0 [g/dm³]:'),
            dcc.Slider(id='cc0_value', min = 0.1, max = 5, step=0.1, value=1,
                      marks={
                          0.5: '0.50',
                          1: '1.00',
                          1.5: '1.50',
                          2: '2.00',
                          2.5: '2.50',
                          3: '3.00',
                          3.5: '3.50',
                          4: '4.00',
                          4.5: '4.50'
                      }),
            html.Div(id='cc0_slider_output')
        ]),
        dbc.FormGroup([
            dbc.Label(r'Concentração Inicial Substrato - Cs0 [g/dm³]:'),
            dcc.Slider(id='cs0_value', min = 0, max = 1250, step=25, value=250,
                      marks={
                          100: '100',
                          250: '250',
                          500: '500',
                          750: '750',
                          1000: '1000',
                          1250: '1250'
                      }),
            html.Div(id='cs0_slider_output')
        ])
    ],
    body = True,
)

controls_time = dbc.Card(
    [
        dbc.FormGroup([
            dbc.Label('Tempo de Batelada - t [h]:'),
            dcc.Slider(id='time_value', min = 6, max = 48, step=1, value=12,
                      marks={
                          6: '6 h',
                          12: '12 h',
                          24: '24 h',
                          36: '36 h'
                      }),
            html.Div(id='time_slider_output')
        ]),
        dcc.Dropdown(
            id='drop_to_plot',
            options=[
                {'label': 'Células', 'value': 'cell'},
                {'label': 'Substrato', 'value': 'substract'},
                {'label': 'Etanol', 'value': 'product'}
            ],
            multi=False,
            value = 'cell'
        ),
        dcc.Graph(id='simul_graph')
    ],
    body = True,
)

In [3]:
# application definition
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

In [4]:
# building application layout
app.layout = dbc.Container([
    html.H1('Simulador de Biorreator - Fermentação de Glicose em Etanol'),
    html.Hr(),
    dbc.Row(
        [
            dbc.Col(controls, md = 4),
            dbc.Col(controls_time, md = 8)
        ],
        align='center',),],
    fluid = True,
)

In [5]:
# define callbacks
@app.callback(Output('K_s_slider_output', 'children'), [Input('ks_value', 'value')])
def update_ks(value):
    return 'Constante de Monod selecionada Ks: %.2f g/dm³'%value

@app.callback(Output('mu_slider_output', 'children'), [Input('mu_value', 'value')])
def update_mu(value):
    return 'Velocidade Máxima selecionada: %.2f 1/h'%value

@app.callback(Output('cc0_slider_output', 'children'), [Input('cc0_value', 'value')])
def update_cc0(value):
    return 'Conc. Inicial de células selecionada: %.2f g/dm³'%value

@app.callback(Output('cs0_slider_output', 'children'), [Input('cs0_value', 'value')])
def update_cs0(value):
    return 'Conc. Inicial de substrato selecionada: %.2f g/dm³'%value

@app.callback(Output('time_slider_output', 'children'), [Input('time_value', 'value')])
def update_time(value):
    return 'Tempo de Batelada selecionado: %.2f h'%value

@app.callback(Output('simul_graph', 'figure'), [Input('prod_value', 'value'), Input('ks_value', 'value'),
                                               Input('mu_value', 'value'), Input('cc0_value', 'value'),
                                               Input('cs0_value', 'value'), Input('time_value', 'value'),
                                               Input('drop_to_plot', 'value')])
def update_plot(prod, Ks, mu, cc0, cs0, tb, option):
    
    # class instance
    br = BioReactor(Cc0=cc0, Cs0=cs0, production=prod*1000, batch_time=tb)
    
    # Monod's parameter setter
    br.change_cells(Ks, mu)
    
    # testing the Runge-Kutta's implementation
    results = br.runge_kutta()
    
    # calculate necessary volume
    br.calculate_volume()
    
    # changing columns names
    cols = ['Tempo de Batelada [h]', 'Conc. Células [g/dm³]', 'Conc. Glicose [g/dm³]', 'Conc. Etanol [g/dm³]']
    results.columns = cols
    
    # option dictionary
    opt = {
        'cell': 'Conc. Células [g/dm³]',
        'substract': 'Conc. Glicose [g/dm³]',
        'product': 'Conc. Etanol [g/dm³]'
    }
    
    # graph title definition
    title = 'Perfil Temporal de %s - Volume do Sistema: %.2f m³'%(opt[option], br.volume)

    # create plot to show
    fig = px.line(results, x = 'Tempo de Batelada [h]', y = opt[option], title = title)
    
    return fig

In [ ]:
# run app
app.run_server(port=1234, debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:1234/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
